In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
#Initialisation for the different algorithms:
barcode1 = np.array([[0, 100.], [1.37, 94.2], [5.80, 86.], [18., 83.75], [42., 60.]])      # 24 permutations (1234)
barcode2 = np.array([[0., 10.], [1., 8.], [2, 9.], [3., 7.]])      # 3 permutations (213)
barcode3 = np.array([[0., 10.], [1., 9.], [2., 8.], [3., 7.], [4., 6.]])        # 24 permutations (1234)
barcode4 = np.array([[0., 100.], [1.37, 94.2], [5.80, 86.], [9.75, 85.], [18., 83.75], [42., 60.]])       #120 permutations (12345)
barcodeE = np.array([[0., 10.], [9., 1.], [6., 5.], [5., 11.]])

#Choose your barcode:
barcode = barcode4
barcode_size = int(barcode.size/2)

In [3]:
#Tests:
error1 = 0; error2 = 0; error3 = 0

for n in range(barcode_size):
    if barcode[n][0] > barcode[n][1]:
        print ("error1: birth need to be lower than death")
        error1 = 1
    
for n in range(barcode_size - 1):
    if barcode[n][0] > barcode[n+1][0]:
        print("error2: we need a sorted by birth barcode.")
        error2 = 1

for n in range(barcode_size - 1):
    if barcode[n+1][1] > barcode[0][1]:
        print("error3: the first bar must contains all the others")
        error3 = 1

if error1 == 0 and error2 == 0 and error3 == 0 :
    print ("Tests: passed")    

Tests: passed


In [4]:
#Auto initialisation with in function of what was token:
matrix_adj = np.zeros((barcode_size, barcode_size))     #empty matrix.
matrix_pb = np.zeros((barcode_size, barcode_size))     #empty matrix.
matrix_real = np.zeros((barcode_size, barcode_size))    #empty matrix.
matrix_fr = np.zeros((barcode_size, barcode_size))    #empty matrix.

permutation_number = 1                                #Number of possible permutations for a barcode.
index = np.linspace(0, 0, barcode_size-1)    
proba = np.linspace(0, 0, barcode_size-1)             #list of all probabilities for each line (equi).
finit_support = np.zeros(barcode_size)                  #lifetimes
for i in range(barcode_size):
    finit_support[i] += barcode[i][1] - barcode[i][0] 
Number = 1000
vector_fr = []

In [5]:
#Permutation Number:
for i in range(barcode_size)[1:]:
    for j in range(barcode_size)[:i]:
        if barcode[j][1] > barcode[i][1]:
            index[i-1] += 1
    permutation_number *= index[i-1]
    
#Equivalent Class:
for i in range(barcode_size - 1):
    index[i] = index[i] - (i+1)        #To know which of them are already in the right order.
clock = 1
while(clock < barcode_size):
    min = 1000
    memory = 0
    for j in range(barcode_size - 1):
        if index[j] < min:
            min = index[j]
            memory = j
    index[memory] = clock
    clock += 1

In [6]:
#Matrix initialisation:

#Probability Matrix:
for i in range(barcode_size):
    for j in range(barcode_size)[i+1:]:
        if barcode[i][1] > barcode[j][1]:
            matrix_pb[j][i] = 1
for x in range(barcode_size)[1:]:
    clock = 0
    for y in range(barcode_size)[:x]:
        clock += matrix_pb[x][y]             #number of 1 on a single line
    probability = 1./clock                   #normalisation for equiprobability
    for z in range(barcode_size)[:x]:
        if matrix_pb[x][z] == 1:
            matrix_pb[x][z] = probability

#Adjacency Matrix:
for k in range(barcode_size):
    matrix_adj[k][k] = barcode[k][0]             
for l in range(barcode_size):
    for m in range(barcode_size)[l+1:]:
        if barcode[l][1] > barcode[m][1]:
            matrix_adj[m][l] = barcode[m][1]

In [7]:
#Possible realisation
for n in range(Number):
    #Real Matrix
    for k in range(barcode_size):
        matrix_real[k][k] = matrix_adj[k][k]   
    for i in range(barcode_size)[1:]:
        clock = 0
        for j in range(barcode_size)[:i]:
            if matrix_pb[i][j] != 0:
                clock += 1
        if clock != 0:
            probability = 1./clock
        else:
            probability = 0.
        proba[i-1] = probability
        complete = False
        nbr = random.random()
        for j in range(barcode_size)[:i]:
            if (nbr < ((j+1)*proba[i-1]) and (not complete)):
                matrix_real[i][j] = matrix_adj[i][j]
                complete = True
            else:
                matrix_real[i][j] = 0
    for o in range(barcode_size)[1:]:
        for p in range(barcode_size)[:o]:
            if matrix_real[o][p] != 0:
                matrix_fr[o][p] += 1

#Normalisation of frequency matrix:
for i in range(barcode_size)[1:]:
    sum = 0
    for j in range(barcode_size)[:i]:
        sum += matrix_fr[i][j]
    for j in range(barcode_size)[:i]:
        matrix_fr[i][j] = matrix_fr[i][j]/sum

print("The number of permutations for this barcode is : ", int(permutation_number))
print("The equivalence class of this barcode is : (", index, "). \n")
print("All possible matrices are in this form: \n", matrix_adj, "\n")
print("The last matrix realised was: \n", matrix_real, "\n")
print("The Probability matrix is: \n", matrix_pb, "\n")
print("The Frequency matrix over 1000 realisations is: \n", matrix_fr, "\n")     

The number of permutations for this barcode is :  120
The equivalence class of this barcode is : ( [1. 2. 3. 4. 5.] ). 

All possible matrices are in this form: 
 [[ 0.    0.    0.    0.    0.    0.  ]
 [94.2   1.37  0.    0.    0.    0.  ]
 [86.   86.    5.8   0.    0.    0.  ]
 [85.   85.   85.    9.75  0.    0.  ]
 [83.75 83.75 83.75 83.75 18.    0.  ]
 [60.   60.   60.   60.   60.   42.  ]] 

The last matrix realised was: 
 [[ 0.    0.    0.    0.    0.    0.  ]
 [94.2   1.37  0.    0.    0.    0.  ]
 [ 0.   86.    5.8   0.    0.    0.  ]
 [ 0.   85.    0.    9.75  0.    0.  ]
 [ 0.    0.   83.75  0.   18.    0.  ]
 [ 0.    0.    0.   60.    0.   42.  ]] 

The Probability matrix is: 
 [[0.         0.         0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.        ]
 [0.5        0.5        0.         0.         0.         0.        ]
 [0.33333333 0.33333333 0.33333333 0.         0.         0.        ]
 [0.25       0.25       0.

In [ ]:
print(matrix_adj, "\n")

def tree_real(matrix_arg, k = 0):
    tree_realizations = np.empty([barcode_size, barcode_size])
    matrix = np.zeros((barcode_size, barcode_size))
    matrix[0] = list[0]
    for i in range(barcode_size)[1:]:
        clock = 0
        compteur = 0
        for j in range(barcode_size)[:i]:
            if list[i][j] != 0:
                clock  +=  1       
        if clock == 1:
            matrix[i] = list[i]        
        if clock > 1:
            for l in range(barcode_size)[:i]:
                if list[i][l] != 0:
                    if (clock - k) > 1:
                        clock -= 1
                    elif (clock - k) > 0:
                        tree_real(list, k+1)
                        matrix[i][l] = list[i][l]
    np.append(tree_realizations, matrix)
    return tree_realizations
    
print(tree_real(matrix_adj))